# Импортирую необходимые библиотеки и загруждаю наш обработанный датасет для обучения модели

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix, make_scorer, average_precision_score
from sklearn.model_selection import cross_val_score

In [10]:
df=pd.read_csv('step_2.csv', low_memory=False)

# Обучение модели

In [13]:
categorical_cols = df.select_dtypes(include=['object']).columns
encoder = OneHotEncoder(sparse_output=False)
encoded_cats = encoder.fit_transform(df[categorical_cols])
encoded_df = pd.DataFrame(encoded_cats, columns=encoder.get_feature_names_out(categorical_cols))
df = pd.concat([df.reset_index(drop=True), encoded_df.reset_index(drop=True)], axis=1)
df.drop(categorical_cols, axis=1, inplace=True)

In [15]:
X = df.drop('target', axis=1)
y = df['target']

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [19]:
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

C:\Users\rzhal\anaconda3\Lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


In [20]:
model = RandomForestClassifier(class_weight='balanced', random_state=42)
model.fit(X_resampled, y_resampled)

RandomForestClassifier(class_weight='balanced', random_state=42)

# Вывод матрицы

In [22]:
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]

roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f'ROC-AUC: {roc_auc:.4f}')
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

ROC-AUC: 1.0000
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       688
           1       1.00      1.00      1.00         4

    accuracy                           1.00       692
   macro avg       1.00      1.00      1.00       692
weighted avg       1.00      1.00      1.00       692

[[688   0]
 [  0   4]]


# Наша модель успешно обучилась на нашей выборке
Проведем еще несколько измерений для того, что бы убедиться что она не была переобучена

In [27]:
# 1. Кросс-валидация
# Создаем модель
model = RandomForestClassifier(random_state=42)

# Используем кросс-валидацию с ROC-AUC в качестве метрики
scores = cross_val_score(model, X, y, cv=5, scoring='roc_auc')

# Выводим результаты
print("ROC-AUC scores for each fold:", scores)
print("Mean ROC-AUC score:", np.mean(scores))

# 2. Изменение метрик: AUC-PR
# Разделяем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Обучаем модель
model.fit(X_train, y_train)

# Получаем предсказанные вероятности для положительного класса
y_scores = model.predict_proba(X_test)[:, 1]

# Вычисляем AUC-PR
auc_pr = average_precision_score(y_test, y_scores)

# Выводим результат
print("AUC-PR:", auc_pr)

ROC-AUC scores for each fold: [1. 1. 1. 1. 1.]
Mean ROC-AUC score: 1.0
AUC-PR: 1.0


In [29]:
# Создаем и обучаем модель
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Оцениваем модель на обучающей выборке
y_train_pred = model.predict_proba(X_train)[:, 1]
train_roc_auc = roc_auc_score(y_train, y_train_pred)
train_auc_pr = average_precision_score(y_train, y_train_pred)

# Оцениваем модель на тестовой выборке
y_test_pred = model.predict_proba(X_test)[:, 1]
test_roc_auc = roc_auc_score(y_test, y_test_pred)
test_auc_pr = average_precision_score(y_test, y_test_pred)

# Выводим результаты
print("Train ROC-AUC:", train_roc_auc)
print("Train AUC-PR:", train_auc_pr)
print("Test ROC-AUC:", test_roc_auc)
print("Test AUC-PR:", test_auc_pr)

# Проверяем на переобучение
if train_roc_auc - test_roc_auc > 0.1 or train_auc_pr - test_auc_pr > 0.1:
    print("Модель может быть переобучена.")
else:
    print("Модель не показывает признаков переобучения.")

Train ROC-AUC: 1.0
Train AUC-PR: 1.0
Test ROC-AUC: 1.0
Test AUC-PR: 1.0
Модель не показывает признаков переобучения.


# Цели достигнуты, подготавливаю модель к локальному запуску